# Setup

In [743]:
import pandas as pd
import pyodbc, os
from ydata_profiling import ProfileReport

In [744]:
sales_conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=..\GO-databases DEDS-week 4 & 5\go_sales.accdb;')
crm_conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=..\GO-databases DEDS-week 4 & 5\go_crm.accdb;')

In [745]:
product = pd.read_sql("SELECT * FROM product", sales_conn)
order_header = pd.read_sql("SELECT * FROM order_header", sales_conn)
order_details = pd.read_sql("SELECT * FROM order_details", sales_conn)
retailer = pd.read_sql("SELECT * FROM retailer", crm_conn)
retailer_site = pd.read_sql("SELECT * FROM crm_retailer_site", crm_conn)
retailer_contact = pd.read_sql("SELECT * FROM retailer_contact", crm_conn)





C:\Users\Wouter Swijnenburg\AppData\Local\Temp\ipykernel_19020\2670134883.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql("SELECT * FROM product", sales_conn)
C:\Users\Wouter Swijnenburg\AppData\Local\Temp\ipykernel_19020\2670134883.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_header = pd.read_sql("SELECT * FROM order_header", sales_conn)
C:\Users\Wouter Swijnenburg\AppData\Local\Temp\ipykernel_19020\2670134883.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_details = pd.read

# COde

In [746]:
numeric_columns = ['PRODUCT_NUMBER', 'PRODUCT_TYPE_CODE', 'PRODUCTION_COST', 'MARGIN']
non_numeric_columns = list(product.drop(numeric_columns, axis = 1).columns)
non_numeric_columns

['INTRODUCTION_DATE',
 'PRODUCT_IMAGE',
 'LANGUAGE',
 'PRODUCT_NAME',
 'DESCRIPTION']

In [747]:
product['INTRODUCTION_DATE'] = product['INTRODUCTION_DATE'].fillna('onbekend')
product['INTRODUCTION_DATE'].isna().sum()

0

In [748]:
modus = product['LANGUAGE'].mode()[0]
product['LANGUAGE'] = product['LANGUAGE'].fillna(modus)
product['LANGUAGE'].isna().sum()


0

In [749]:
subset_product_name_contains_granite = product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), :]
subset_product_name_contains_granite = product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), :]
product.loc[(~product['PRODUCT_NAME'].isna()) & (product['PRODUCT_NAME'].str.contains('Granite')), 'PRODUCT_TYPE_CODE'] = subset_product_name_contains_granite['PRODUCT_TYPE_CODE'].fillna(9)

In [750]:
product['INTRODUCTION_YEAR'] = product['INTRODUCTION_DATE'].str[-4:]
subset_introduction_year_2011_2019 = product.loc[(product['INTRODUCTION_YEAR'] == '2011') | (product['INTRODUCTION_YEAR'] == '2019'), :]
grouped_subset_introduction_year_2011_2019 = subset_introduction_year_2011_2019.groupby(['INTRODUCTION_YEAR', 'PRODUCT_TYPE_CODE'])['PRODUCT_NUMBER'].count().reset_index()
grouped_subset_introduction_year_2011_2019 = grouped_subset_introduction_year_2011_2019.rename(columns = {'PRODUCT_NUMBER' : 'Aantal producten'})
grouped_subset_introduction_year_2011_2019.sort_values(['INTRODUCTION_YEAR', 'Aantal producten'], ascending = [True, False])
product.loc[(product['INTRODUCTION_YEAR'] == '2011'), 'PRODUCT_TYPE_CODE'] = product.loc[(product['INTRODUCTION_YEAR'] == '2011'), 'PRODUCT_TYPE_CODE'].fillna(1)
product.loc[(product['INTRODUCTION_YEAR'] == '2019'), 'PRODUCT_TYPE_CODE'] = product.loc[(product['INTRODUCTION_YEAR'] == '2019'), 'PRODUCT_TYPE_CODE'].fillna(19)
product['PRODUCT_TYPE_CODE'].isna().sum()

0

### Product Categorisch

 Remove all unused columns

In [751]:
product = product.dropna()

# product = product.drop('INTRODUCTION_DATE', axis='columns')
product = product.drop('INTRODUCTION_YEAR', axis='columns')
# product = product.drop('PRODUCT_IMAGE', axis='columns')
# product = product.drop('PRODUCT_NAME', axis='columns')
# product = product.drop('PRODUCT_TYPE_CODE', axis='columns')
# product = product.drop('DESCRIPTION', axis='columns')   

# product = product.drop('PRODUCTION_COST', axis='columns')
# product = product.drop('MARGIN', axis='columns')

product

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION
0,53,10-1-2021,8,28.55,.28,P53ME2CA8.jpg,EN,Firefly Climbing Lamp,This climbing lamp is water-resistant. It come...
1,54,10-1-2021,8,35.33,.33,P54ME2CA8.jpg,EN,Firefly Charger,Power charger for the Firefly Climbing Lamp. T...
2,55,onbekend,8,4,.5,P55ME2CA8.jpg,EN,Firefly Rechargeable Battery,Rechargeable battery pack for the Firefly Clim...
3,56,10-1-2021,8,9,.5,P56ME2CA8.jpg,EN,Granite Chalk Bag,Nylon web bag perfect for holding several stic...
4,57,10-1-2021,9,40,.5,P57ME2TL9.jpg,EN,Granite Ice,A general purpose mountaineering axe for snow ...
...,...,...,...,...,...,...,...,...,...
108,106,5-12-2019,19,536.09,.43,P106GE5WD19.jpg,EN,Hailstorm Steel Woods Set,These steel woods are manufactured to exact to...
109,107,13-1-2020,19,509.07,.55,P105GE5WD19.jpg,EN,Lady Hailstorm Titanium Woods Set,Crafted of 6-4 titanium which is 40% lighter a...
111,109,10-12-2019,20,35,.43,P109GE5PT20.jpg,EN,Course Pro Putter,High quality putter is cast from a soft yello...
112,110,Blue Steel Putter,20,60,.5,P110GE5PT20.jpg,EN,10-12-2019,This premium putter has copper mixed with ligh...


Verwisselde troep

In [752]:
product.dtypes

numeric_columns = ['PRODUCT_NUMBER', 'PRODUCT_TYPE_CODE', 'PRODUCTION_COST', 'MARGIN']
non_numeric_columns = list(product.drop(numeric_columns, axis = 1).columns)

wrong_index_dict = dict()

for index in product.index:
    wrong_index_dict[index] = []

for numeric_column in numeric_columns:
    #Stringwaarden vinden in numerieke kolommen:
    wrong_rows = product.loc[pd.to_numeric(product[numeric_column], errors = 'coerce').isnull(), [numeric_column]]
    
    for index, row in wrong_rows.iterrows():
        wrong_index_dict[index].append(numeric_column)
    
for non_numeric_column in non_numeric_columns:
    #Numerieke waarden vinden in stringkolommen
    wrong_rows = product.loc[~pd.to_numeric(product[non_numeric_column], errors = 'coerce').isnull(), [non_numeric_column]]
    
    for index, row in wrong_rows.iterrows():
        wrong_index_dict[index].append(non_numeric_column)

wrong_index_dict


{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 15: [],
 16: [],
 17: [],
 18: ['PRODUCTION_COST', 'DESCRIPTION'],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 31: [],
 32: [],
 33: [],
 34: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: ['PRODUCTION_COST', 'PRODUCT_IMAGE'],
 45: [],
 46: [],
 47: ['PRODUCTION_COST', 'LANGUAGE'],
 48: [],
 49: ['PRODUCTION_COST', 'DESCRIPTION'],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 56: [],
 57: ['MARGIN', 'PRODUCT_NAME'],
 58: [],
 59: [],
 60: [],
 61: [],
 63: [],
 64: [],
 65: [],
 66: [],
 68: ['MARGIN', 'INTRODUCTION_DATE'],
 69: ['PRODUCTION_COST', 'DESCRIPTION'],
 70: [],
 71: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [],
 78: ['PRODUCTION_COST', 'DESCRIPTION'],
 80: [],
 81: [],
 82: [],
 83: [],
 84: ['MARGIN', 'INTRODUCTION_DATE'],
 85: ['PRODUCTION_COST', 'PRODUCT_NAME'],
 86: [],
 87: [],
 8

In [753]:
for key_value_pair in list(wrong_index_dict.items()):
    
    if(key_value_pair[1]):
        index = key_value_pair[0]
        kolom1 = key_value_pair[1][0]
        kolom2 = key_value_pair[1][1]
        
        product.loc[product.index == index, [kolom1, kolom2]] = product.loc[product.index == index, [kolom2, kolom1]].values

product

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION
0,53,10-1-2021,8,28.55,.28,P53ME2CA8.jpg,EN,Firefly Climbing Lamp,This climbing lamp is water-resistant. It come...
1,54,10-1-2021,8,35.33,.33,P54ME2CA8.jpg,EN,Firefly Charger,Power charger for the Firefly Climbing Lamp. T...
2,55,onbekend,8,4,.5,P55ME2CA8.jpg,EN,Firefly Rechargeable Battery,Rechargeable battery pack for the Firefly Clim...
3,56,10-1-2021,8,9,.5,P56ME2CA8.jpg,EN,Granite Chalk Bag,Nylon web bag perfect for holding several stic...
4,57,10-1-2021,9,40,.5,P57ME2TL9.jpg,EN,Granite Ice,A general purpose mountaineering axe for snow ...
...,...,...,...,...,...,...,...,...,...
108,106,5-12-2019,19,536.09,.43,P106GE5WD19.jpg,EN,Hailstorm Steel Woods Set,These steel woods are manufactured to exact to...
109,107,13-1-2020,19,509.07,.55,P105GE5WD19.jpg,EN,Lady Hailstorm Titanium Woods Set,Crafted of 6-4 titanium which is 40% lighter a...
111,109,10-12-2019,20,35,.43,P109GE5PT20.jpg,EN,Course Pro Putter,High quality putter is cast from a soft yello...
112,110,Blue Steel Putter,20,60,.5,P110GE5PT20.jpg,EN,10-12-2019,This premium putter has copper mixed with ligh...


In [754]:
product['PRODUCT_NUMBER'] = product['PRODUCT_NUMBER'].astype(int)
product['PRODUCT_TYPE_CODE'] = product['PRODUCT_NUMBER'].astype(int)
product['PRODUCTION_COST'] = product['PRODUCTION_COST'].astype(float)
product['MARGIN'] = product['MARGIN'].astype(float)

product.dtypes

PRODUCT_NUMBER         int32
INTRODUCTION_DATE     object
PRODUCT_TYPE_CODE      int32
PRODUCTION_COST      float64
MARGIN               float64
PRODUCT_IMAGE         object
LANGUAGE              object
PRODUCT_NAME          object
DESCRIPTION           object
dtype: object

## Order Details Categorisch

In [755]:
order_details = order_details.dropna()
order_details

,ORDER_DETAIL_CODE,ORDER_NUMBER,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,78.37,1258,17,92.0,60.29,84.41,48616
2,48618,1544,17,82.0,60.29,84.41,78.37
3,48619,1249,17,190.0,60.29,84.41,72.26
4,48620,1268,17,152.0,60.29,84.41,78.05
5,48621,1266,17,12.0,60.29,84.41,84.41
...,...,...,...,...,...,...,...
43058,102951,8614,11,16.0,250,350,325
43059,102952,8611,11,18.0,250,350,350
43060,102953,8591,11,18.0,250,350,325
43061,102954,8619,11,18.0,250,350,350


Select correct data types

In [756]:
order_details['UNIT_PRICE'] = order_details['UNIT_PRICE'].astype(float)
order_details['UNIT_SALE_PRICE'] = order_details['UNIT_PRICE'].astype(float)
order_details['PRODUCT_NUMBER'] = order_details['PRODUCT_NUMBER'].astype(int)
order_details['QUANTITY'] = order_details['QUANTITY'].astype(int)
order_details['UNIT_COST'] = order_details['UNIT_COST'].astype(float)
order_details['ORDER_NUMBER'] = order_details['ORDER_NUMBER'].astype(int)
order_details['ORDER_DETAIL_CODE'] = order_details['ORDER_DETAIL_CODE'].astype(float).astype(int)
order_details.dtypes

ORDER_DETAIL_CODE      int32
ORDER_NUMBER           int32
PRODUCT_NUMBER         int32
QUANTITY               int32
UNIT_COST            float64
UNIT_PRICE           float64
UNIT_SALE_PRICE      float64
dtype: object

## Order Header Categorisch

Remove unused data

In [757]:
order_header = order_header.drop('ORDER_METHOD_CODE', axis='columns')
order_header = order_header.drop('SALES_BRANCH_CODE', axis='columns')
order_header = order_header.drop('SALES_STAFF_CODE', axis='columns')
order_header

,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,ORDER_DATE
0,1831,Sport & Freizeit,36,52,2022-11-11
1,1832,Sport & Freizeit,38,43,2022-11-19
2,1833,Sport & Freizeit,40,60,2022-11-11
3,1834,Die Zeltstadt,47,40,2022-11-15
4,1835,Die Zeltstadt,48,57,2022-12-14
...,...,...,...,...,...
5355,9238,Fu Chin Sport Equipment Co.,362,353,2021-10-17
5356,9239,"Chen Yu Enterprise Co.,",365,356,2022-02-16
5357,9240,"Chen Yu Enterprise Co.,",365,356,2022-10-12
5358,9241,"Chen Yu Enterprise Co.,",363,354,2022-01-20


Fill in unknown/impossible to know data

In [758]:

order_header['RETAILER_NAME'].isna().sum()
order_header['RETAILER_NAME'] = order_header['RETAILER_NAME'].fillna('Unknown')
order_header['RETAILER_NAME'].isna().sum()

order_header['ORDER_DATE'].isna().sum()
order_header['ORDER_DATE'] = order_header['ORDER_DATE'].fillna('Unknown')
order_header['ORDER_DATE'].isna().sum()

0

Fix the retailer name/ contact code switch

In [759]:
numeric_columns = ['RETAILER_CONTACT_CODE']
non_numeric_columns = list()
non_numeric_columns.append("RETAILER_NAME")
non_numeric_columns

wrong_index_dict = dict()

for index in order_header.index:
    wrong_index_dict[index] = []

wrong_index_dict

for numeric_column in numeric_columns:
    #Stringwaarden vinden in numerieke kolommen:
    wrong_rows = order_header.loc[pd.to_numeric(order_header[numeric_column], errors = 'coerce').isnull(), [numeric_column]]
    
    for index, row in wrong_rows.iterrows():
        wrong_index_dict[index].append(numeric_column)
    
for non_numeric_column in non_numeric_columns:
    #Numerieke waarden vinden in stringkolommen
    wrong_rows = order_header.loc[~pd.to_numeric(order_header[non_numeric_column], errors = 'coerce').isnull(), [non_numeric_column]]
    
    for index, row in wrong_rows.iterrows():
        wrong_index_dict[index].append(non_numeric_column)
    

for key_value_pair in list(wrong_index_dict.items()):
    try:
        if(key_value_pair[1]): #Als er minstens 1 item gevuld is op de index...
            index = key_value_pair[0]
            kolom1 = key_value_pair[1][0]
            kolom2 = key_value_pair[1][1]
            
            order_header.loc[order_header.index == index, [kolom1, kolom2]] = order_header.loc[order_header.index == index, [kolom2, kolom1]].values
    except:
        continue
order_header

,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,ORDER_DATE
0,1831,Sport & Freizeit,36,52,2022-11-11 00:00:00
1,1832,Sport & Freizeit,38,43,2022-11-19 00:00:00
2,1833,Sport & Freizeit,40,60,2022-11-11 00:00:00
3,1834,Die Zeltstadt,47,40,2022-11-15 00:00:00
4,1835,Die Zeltstadt,48,57,2022-12-14 00:00:00
...,...,...,...,...,...
5355,9238,Fu Chin Sport Equipment Co.,362,353,2021-10-17 00:00:00
5356,9239,"Chen Yu Enterprise Co.,",365,356,2022-02-16 00:00:00
5357,9240,"Chen Yu Enterprise Co.,",365,356,2022-10-12 00:00:00
5358,9241,"Chen Yu Enterprise Co.,",363,354,2022-01-20 00:00:00


Buuranalyse Retailer site/contact codes

In [760]:
# Kijk naar andere waardes met dezelfde retailer naam & gebruik de meest voorkomende codes
# De overgebleven waardes zijn velden die geen nulls hadden maar foute data als name/code

Retailer_Names = order_header['RETAILER_NAME'].unique()
for company in Retailer_Names:
    subset = order_header.loc[order_header['RETAILER_NAME'].str.contains(company), :]
    if subset['RETAILER_SITE_CODE'].mode().size > 0:
        mode = subset['RETAILER_SITE_CODE'].mode()[0]
        order_header.loc[(order_header['RETAILER_NAME'].str.contains(company)), 'RETAILER_SITE_CODE'] = order_header.loc[(order_header['RETAILER_NAME'] == company), 'RETAILER_SITE_CODE'].fillna(mode)

order_header.loc[order_header['RETAILER_SITE_CODE'].isna(), :]

# Find the most used codes for a specific retailer
# subset = order_header.loc[order_header['RETAILER_NAME'].str.contains('Sport & Freizeit'), :]
# mode = subset['RETAILER_SITE_CODE'].mode()[0]
# mode
# order_header['RETAILER_CONTACT_CODE'].isna().sum()


,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,ORDER_DATE
5173,9039,388,None,396,2021-04-10 00:00:00


Convert to correct datatype

In [761]:
order_header = order_header.dropna()
order_header['ORDER_NUMBER'] = order_header['ORDER_NUMBER'].astype(int)
order_header['RETAILER_SITE_CODE'] = order_header['RETAILER_SITE_CODE'].astype(int)
#order_header['RETAILER_CONTACT_CODE'] = order_header['RETAILER_CONTACT_CODE'].astype(int)
order_header['ORDER_DATE'].replace("Unknown", None)
order_header = order_header.dropna()


pd.to_datetime(order_header['ORDER_DATE'])
order_header.dtypes

Unexpected exception formatting exception. Falling back to standard exception


C:\Users\Wouter Swijnenburg\AppData\Local\Temp\ipykernel_19020\1063549907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_header['ORDER_NUMBER'] = order_header['ORDER_NUMBER'].astype(int)
C:\Users\Wouter Swijnenburg\AppData\Local\Temp\ipykernel_19020\1063549907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_header['RETAILER_SITE_CODE'] = order_header['RETAILER_SITE_CODE'].astype(int)
Traceback (most recent call last):
  File "pandas\_libs\tslib.pyx", line 605, in pandas._libs.tslib.arra

## Retailer

Remove unused columns

In [762]:
retailer = retailer.drop('RETAILER_TYPE_CODE', axis='columns')
retailer = retailer.drop('RETAILER_CODEMR', axis='columns')
retailer


,RETAILER_CODE,COMPANY_NAME
0,1,ActiForme
1,15,SportsClub
2,23,Anapurna
3,32,Cordages Discount
4,33,Altitudes extrêmes
...,...,...
104,193,Allo Allo
105,194,Caravanserai
106,209,Ausrüstungshaus Globetrotter
107,210,Golfer-Paradies


Fill blank company names

In [763]:
retailer['COMPANY_NAME'] = retailer['COMPANY_NAME'].fillna('Unknown')
retailer.loc[retailer['COMPANY_NAME'].isna(), :]


,RETAILER_CODE,COMPANY_NAME


## Retailer Contact

Remove unused columns

In [764]:
retailer_contact = retailer_contact.drop('FIRST_NAME', axis='columns')
retailer_contact = retailer_contact.drop('LAST_NAME', axis='columns')
retailer_contact = retailer_contact.drop('JOB_POSITION_EN', axis='columns')
retailer_contact = retailer_contact.drop('EXTENSION', axis='columns')
retailer_contact = retailer_contact.drop('FAX', axis='columns')
retailer_contact = retailer_contact.drop('E_MAIL', axis='columns')
retailer_contact = retailer_contact.drop('GENDER', axis='columns')


retailer_contact


,RETAILER_CONTACT_CODE,RETAILER_SITE_CODE
0,399,391
1,400,392
2,401,393
3,402,394
4,403,395
...,...,...
386,386,394
387,395,387
388,396,388
389,397,389


Fill in missing site codes

In [765]:
# Look to next & previous site code, if the value between these codes doesn't exist, fill oftewel buuranalyse
retailer_contact.loc[retailer_contact['RETAILER_SITE_CODE'].isna(), :]
retailer_contact['RETAILER_SITE_CODE'] = retailer_contact['RETAILER_SITE_CODE'].fillna('Unknown')

# For the time being set them as unknown

# Rapportages

In [766]:

profile = ProfileReport(product, title = 'Kwaliteitsrapportage product')
try:
    os.remove("product.html")
except:
    print("tu tu tu tu tututu")
    
profile.to_file("product.html")
product

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION
0,53,10-1-2021,53,28.55,0.28,P53ME2CA8.jpg,EN,Firefly Climbing Lamp,This climbing lamp is water-resistant. It come...
1,54,10-1-2021,54,35.33,0.33,P54ME2CA8.jpg,EN,Firefly Charger,Power charger for the Firefly Climbing Lamp. T...
2,55,onbekend,55,4.00,0.50,P55ME2CA8.jpg,EN,Firefly Rechargeable Battery,Rechargeable battery pack for the Firefly Clim...
3,56,10-1-2021,56,9.00,0.50,P56ME2CA8.jpg,EN,Granite Chalk Bag,Nylon web bag perfect for holding several stic...
4,57,10-1-2021,57,40.00,0.50,P57ME2TL9.jpg,EN,Granite Ice,A general purpose mountaineering axe for snow ...
...,...,...,...,...,...,...,...,...,...
108,106,5-12-2019,106,536.09,0.43,P106GE5WD19.jpg,EN,Hailstorm Steel Woods Set,These steel woods are manufactured to exact to...
109,107,13-1-2020,107,509.07,0.55,P105GE5WD19.jpg,EN,Lady Hailstorm Titanium Woods Set,Crafted of 6-4 titanium which is 40% lighter a...
111,109,10-12-2019,109,35.00,0.43,P109GE5PT20.jpg,EN,Course Pro Putter,High quality putter is cast from a soft yello...
112,110,Blue Steel Putter,110,60.00,0.50,P110GE5PT20.jpg,EN,10-12-2019,This premium putter has copper mixed with ligh...


In [767]:
# This takes like 30 mins & the html won't be able to open in your lifetime
# profile = ProfileReport(order_details, title = 'Kwaliteitsrapportage product', dark_mode= True)
# try:
#     os.remove("order_details.html")
# except:
#     print("tu tu tu tu tututu")
    
# profile.to_file("order_details.html")
order_details

,ORDER_DETAIL_CODE,ORDER_NUMBER,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,78,1258,17,92,60.29,84.41,84.41
2,48618,1544,17,82,60.29,84.41,84.41
3,48619,1249,17,190,60.29,84.41,84.41
4,48620,1268,17,152,60.29,84.41,84.41
5,48621,1266,17,12,60.29,84.41,84.41
...,...,...,...,...,...,...,...
43058,102951,8614,11,16,250.00,350.00,350.00
43059,102952,8611,11,18,250.00,350.00,350.00
43060,102953,8591,11,18,250.00,350.00,350.00
43061,102954,8619,11,18,250.00,350.00,350.00


In [768]:
# profile = ProfileReport(order_header, title = 'Kwaliteitsrapportage product', dark_mode= True)
# try:
#     os.remove("order_header.html")
# except:
#     print("tu tu tu tu tututu")
    
# profile.to_file("order_header.html")
order_header

,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,ORDER_DATE
0,1831,Sport & Freizeit,36,52,2022-11-11 00:00:00
1,1832,Sport & Freizeit,38,43,2022-11-19 00:00:00
2,1833,Sport & Freizeit,40,60,2022-11-11 00:00:00
3,1834,Die Zeltstadt,47,40,2022-11-15 00:00:00
4,1835,Die Zeltstadt,48,57,2022-12-14 00:00:00
...,...,...,...,...,...
5355,9238,Fu Chin Sport Equipment Co.,362,353,2021-10-17 00:00:00
5356,9239,"Chen Yu Enterprise Co.,",365,356,2022-02-16 00:00:00
5357,9240,"Chen Yu Enterprise Co.,",365,356,2022-10-12 00:00:00
5358,9241,"Chen Yu Enterprise Co.,",363,354,2022-01-20 00:00:00


In [769]:
profile = ProfileReport(retailer, title = 'Kwaliteitsrapportage product', dark_mode= True)
try:
    os.remove("retailer.html")
except:
    print("tu tu tu tu tututu")
    
profile.to_file("retailer.html")
retailer

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

,RETAILER_CODE,COMPANY_NAME
0,1,ActiForme
1,15,SportsClub
2,23,Anapurna
3,32,Cordages Discount
4,33,Altitudes extrêmes
...,...,...
104,193,Allo Allo
105,194,Caravanserai
106,209,Ausrüstungshaus Globetrotter
107,210,Golfer-Paradies


In [770]:
profile = ProfileReport(retailer_contact, title = 'Kwaliteitsrapportage product', dark_mode= True)
try:
    os.remove("retailer_contact.html")
except:
    print("tu tu tu tu tututu")
    
profile.to_file("retailer_contact.html")
retailer_contact

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

,RETAILER_CONTACT_CODE,RETAILER_SITE_CODE
0,399,391
1,400,392
2,401,393
3,402,394
4,403,395
...,...,...
386,386,394
387,395,387
388,396,388
389,397,389
